# Phân tích bộ dữ liệu `Heart Disease`


## 1. Preparing datasets

In [1]:
# %pip install numpy matplotlib seaborn graphviz

# --- Thư viện cơ bản ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import graphviz
import os

# --- Thư viện cho Machine Learning ---
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# --- Định nghĩa các đường dẫn ---
# Đi từ `notebooks` ra ngoài một cấp để thấy các thư mục khác
DATA_PATH = '../data/heart_disease.csv'
OUTPUT_DIR = '../outputs/heart_disease/'
# Tạo thư mục output nếu nó chưa tồn tại
os.makedirs(OUTPUT_DIR, exist_ok=True)

# (Tùy chọn) Thêm thư mục src vào path để import các hàm từ utils.py
import sys
sys.path.append('../')
from src.utils import plot_label_distribution, plot_label_original, build_decision_tree, evaluating_decision_tree_class # Ví dụ nếu bạn có hàm riêng

from sklearn.impute import SimpleImputer

# --- Tải và khám phá dữ liệu ---
df = pd.read_csv(DATA_PATH)

# Chuyển cột 'num' thành nhãn nhị phân (0: không bệnh, 1: có bệnh)
df['num'] = (df['num'] > 0).astype(int)

# 1. Tách features và labels
features = df.drop('num', axis=1)
labels = df['num']

# 2. Áp dụng One-Hot Encoding
print("Áp dụng One-Hot Encoding...")
features_encoded = pd.get_dummies(features)
print(f"Số lượng features sau One-Hot Encoding: {features_encoded.shape[1]}")

# 3. Xử lý giá trị thiếu (NaN)
print("\nXử lý các giá trị bị thiếu...")
imputer = SimpleImputer(strategy='median')
# Dùng features_final để lưu lại kết quả cuối cùng
features_final = pd.DataFrame(imputer.fit_transform(features_encoded), columns=features_encoded.columns)

# 4. Chia dữ liệu train/test (SỬ DỤNG features_final)
print("\nChia dữ liệu thành tập train và test...")

# Phân chia train và test theo tỷ lệ
feature_train_60, feature_test_40, label_train_60, label_test_40 = train_test_split(
        features_final, labels, test_size=0.4, shuffle=True, stratify=labels, random_state=42)

feature_train_40, feature_test_60, label_train_40, label_test_60 = train_test_split(
        features_final, labels, test_size=0.6, shuffle=True, stratify=labels, random_state=42)

feature_train_80, feature_test_20, label_train_80, label_test_20 = train_test_split(
        features_final, labels, test_size=0.2, shuffle=True, stratify=labels, random_state=42)

feature_train_90, feature_test_10, label_train_90, label_test_10 = train_test_split(
        features_final, labels, test_size=0.1, shuffle=True, stratify=labels, random_state=42)

#Tạo biểu đồ ban đầu
print("Đang tạo biểu đồ gốc...")
plot_label_original(labels, OUTPUT_DIR)

print("Đang tạo các biểu đồ phân phối lớp...")
# Tạo các biểu đồ được phân chia
plot_label_distribution(label_train_60, label_test_40, 60, 40, OUTPUT_DIR)
plot_label_distribution(label_train_40, label_test_60, 40, 60, OUTPUT_DIR)
plot_label_distribution(label_train_80, label_test_20, 80, 20, OUTPUT_DIR)
plot_label_distribution(label_train_90, label_test_10, 90, 10, OUTPUT_DIR)

print("Hoàn tất xử lý dữ liệu!")

Áp dụng One-Hot Encoding...
Số lượng features sau One-Hot Encoding: 30

Xử lý các giá trị bị thiếu...

Chia dữ liệu thành tập train và test...
Đang tạo biểu đồ gốc...
Biểu đồ đã được lưu tại: ../outputs/heart_disease/charts/class_original.png
Đang tạo các biểu đồ phân phối lớp...
Biểu đồ đã được lưu tại: ../outputs/heart_disease/charts/class_distribution_60_40.png
Biểu đồ đã được lưu tại: ../outputs/heart_disease/charts/class_distribution_40_60.png
Biểu đồ đã được lưu tại: ../outputs/heart_disease/charts/class_distribution_80_20.png
Biểu đồ đã được lưu tại: ../outputs/heart_disease/charts/class_distribution_90_10.png
Hoàn tất xử lý dữ liệu!


## 2. Building the decision tree classifiers


In [2]:
print("Đang tạo confusion matrix 60/40...")
clf_60_40 = build_decision_tree(features_final, feature_train_60, label_train_60, 60, 40, OUTPUT_DIR)

print("Đang tạo confusion matrix 40/60...")
clf_40_60 = build_decision_tree(features_final, feature_train_40, label_train_40, 40, 60, OUTPUT_DIR)

print("Đang tạo confusion matrix 80/20...")
clf_80_20 = build_decision_tree(features_final, feature_train_80, label_train_80, 80, 20, OUTPUT_DIR)

print("Đang tạo confusion matrix 90/10...")
clf_90_10 = build_decision_tree(features_final, feature_train_90, label_train_90, 90, 10, OUTPUT_DIR)

Đang tạo confusion matrix 60/40...
Đã tạo decision tree tại ../outputs/heart_disease/trees/decision_tree_60_40
Đang tạo confusion matrix 40/60...
Đã tạo decision tree tại ../outputs/heart_disease/trees/decision_tree_40_60
Đang tạo confusion matrix 80/20...
Đã tạo decision tree tại ../outputs/heart_disease/trees/decision_tree_80_20
Đang tạo confusion matrix 90/10...
Đã tạo decision tree tại ../outputs/heart_disease/trees/decision_tree_90_10


## 3. Evaluating the decision tree classifiers


In [3]:
print("Đang tạo Classification Report và Confusion Matrix cho test 60")
evaluating_decision_tree_class(clf_40_60, feature_test_60, label_test_60, 40, 60, OUTPUT_DIR)

print("Đang tạo Classification Report và Confusion Matrix cho test 40")
evaluating_decision_tree_class(clf_60_40, feature_test_40, label_test_40, 60, 40, OUTPUT_DIR)

print("Đang tạo Classification Report và Confusion Matrix cho test 20")
evaluating_decision_tree_class(clf_80_20, feature_test_20, label_test_20, 80, 20, OUTPUT_DIR)

print("Đang tạo Classification Report và Confusion Matrix cho test 10")
evaluating_decision_tree_class(clf_90_10, feature_test_10, label_test_10, 90, 10, OUTPUT_DIR)

Đang tạo Classification Report và Confusion Matrix cho test 60
Classification Report (40/60) đã được lưu tại: ../outputs/heart_disease/reports//classification_report_40_60.txt
Confusion Matrix (Depth=12, 40/60 Split) đã được lưu tại: ../outputs/heart_disease/matrices/confusion_matrix_40_60.png
Đang tạo Classification Report và Confusion Matrix cho test 40
Classification Report (60/40) đã được lưu tại: ../outputs/heart_disease/reports//classification_report_60_40.txt
Confusion Matrix (Depth=17, 60/40 Split) đã được lưu tại: ../outputs/heart_disease/matrices/confusion_matrix_60_40.png
Đang tạo Classification Report và Confusion Matrix cho test 20
Classification Report (80/20) đã được lưu tại: ../outputs/heart_disease/reports//classification_report_80_20.txt
Confusion Matrix (Depth=16, 80/20 Split) đã được lưu tại: ../outputs/heart_disease/matrices/confusion_matrix_80_20.png
Đang tạo Classification Report và Confusion Matrix cho test 10
Classification Report (90/10) đã được lưu tại: ../ou

## 4. The depth and accuracy of a decision tree
